In [ ]:
from epw import epw

In [ ]:
import pandas as pd
CRC_Data = pd.read_csv('CRC_REAL.csv')

In [ ]:
new_column_names = ["Hour", "Temperature", "Dew Point", "Humidity", "Wind", 
                    "Speed", "Gust", "Pressure", "Precip.Rate.", 
                    "Precip.Accum.", "UV", "Solar"]
CRC_Data.columns = new_column_names + [CRC_Data.columns[-1]]

In [ ]:
for column in CRC_Data.columns:
    if column not in ["Hour", "Wind", "Date"]:
        CRC_Data[column] = CRC_Data[column].astype(str).str.replace(r'[^\d.]', '', regex=True)

In [ ]:
CRC_Data = CRC_Data.drop(index=[0, 1])
CRC_Data = CRC_Data.reset_index(drop=True)


In [ ]:
# Convert 'Temperature' column from Fahrenheit to Celsius
CRC_Data['Temperature'] = pd.to_numeric(CRC_Data['Temperature'], errors='coerce')
CRC_Data['Temperature'] = (CRC_Data['Temperature'] - 32) * 5.0 / 9.0

# Convert 'Dew Point' column from Fahrenheit to Celsius
CRC_Data['Dew Point'] = pd.to_numeric(CRC_Data['Dew Point'], errors='coerce')
CRC_Data['Dew Point'] = (CRC_Data['Dew Point'] - 32) * 5.0 / 9.0

# Define wind direction to degrees mapping
wind_to_degrees = {
    'North': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'East': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'South': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'West': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5
}

# Convert 'Wind' column from directions to degrees
CRC_Data['Wind'] = CRC_Data['Wind'].map(wind_to_degrees)

# Convert 'Speed' column to numeric and handle non-numeric values
CRC_Data['Speed'] = pd.to_numeric(CRC_Data['Speed'], errors='coerce')

# Fill any NaN values in 'Speed' with 0 or drop them if necessary
CRC_Data['Speed'] = CRC_Data['Speed'].fillna(0)  # Use .dropna() if you prefer to remove rows

# Convert wind speed from m/s to km/h
conversion_factor = 0.44704
CRC_Data['Speed'] = CRC_Data['Speed'] * conversion_factor

# Convert 'Pressure', 'Precip.Rate.', and 'Precip.Accum.' from inches to meters
inch_to_meter = 0.0254
CRC_Data['Precip.Rate.'] = pd.to_numeric(CRC_Data['Precip.Rate.'], errors='coerce') * inch_to_meter
CRC_Data['Precip.Accum.'] = pd.to_numeric(CRC_Data['Precip.Accum.'], errors='coerce') * inch_to_meter

In [ ]:
CRC_Data

In [ ]:
testing = CRC_Data.copy()

In [ ]:
testing['Datetime'] = pd.to_datetime(testing['Date'] + ' ' + testing['Hour'], format='%Y-%m-%d %I:%M %p', errors='coerce')

testing['Year'] = testing['Datetime'].dt.year
testing['Month'] = testing['Datetime'].dt.month
testing['Day'] = testing['Datetime'].dt.day

testing['Hour'] = testing['Datetime'].dt.hour

testing['Hour'] = testing['Hour'] + 1

In [ ]:
#set index as the date
testing.set_index('Datetime', inplace=True)

numeric_columns = ['Temperature', 'Dew Point', 'Wind', 'Speed', 'Pressure', 'Precip.Rate.', 'Precip.Accum.']
for col in numeric_columns:
    testing[col] = pd.to_numeric(testing[col], errors='coerce')
    
#these 4 kept disappearing
for col in ['Humidity', 'Gust', 'UV', 'Solar']:
    testing[col] = pd.to_numeric(testing[col], errors='coerce')

# Resample by the specified length and calculate the average for each period
resample_len = 1
hourly_data = testing.resample(f'{resample_len}H').mean()

full_date_range = pd.date_range(start=testing.index.min(), end=testing.index.max(), freq='H')
full_date_range_df = pd.DataFrame(index=full_date_range)

#left join
hourly_data = full_date_range_df.join(hourly_data, how='left')

# Fill NaNs with the mean of each column
hourly_data.fillna(hourly_data.mean(), inplace=True)

# Reset index to turn 'Datetime' back into a column
hourly_data.reset_index(inplace=True)
hourly_data.rename(columns={'index': 'Datetime'}, inplace=True)

# Adjust the hour to start from 1 instead of 0
hourly_data['Hour'] = hourly_data['Datetime'].dt.hour + 1

# Add 'Date' column based on 'Datetime'
hourly_data['Date'] = hourly_data['Datetime'].dt.date

# Reorder columns if necessary
columns_order = ['Datetime', 'Hour', 'Temperature', 'Dew Point', 'Humidity', 'Wind', 'Speed', 'Gust', 'Pressure', 'Precip.Rate.', 'Precip.Accum.', 'UV', 'Solar', 'Date']
# Only include columns that are present in hourly_data
columns_order = [col for col in columns_order if col in hourly_data.columns]

# Reorder DataFrame
hourly_data = hourly_data[columns_order]

print(hourly_data.head())

In [ ]:
hourly_data = hourly_data.round(1)
hourly_data

In [ ]:
a = epw()
a.read('EPW_Files/USA_GA_Atlanta-Hartsfield-Jackson.Intl.AP.722190_TMY3.epw')
test_df = a.dataframe
comparison = a.dataframe

In [ ]:
# Define the custom conversion function
def convert_inhg_to_pa(series):
    return series * 3386.39

In [ ]:
test_df['Year'] = 2023
test_df['Dry Bulb Temperature'] = hourly_data['Temperature']
test_df['Dew Point Temperature'] = hourly_data['Dew Point']
test_df['Relative Humidity'] = hourly_data['Humidity']#.astype(int)
test_df['Wind Direction'] = hourly_data['Wind']
test_df['Wind Speed'] = hourly_data['Speed']
# Convert the 'Pressure_inHg' column to Pascals
test_df['Atmospheric Station Pressure'] = hourly_data['Pressure'].apply(convert_inhg_to_pa)

In [ ]:
id = 14

print(test_df.columns[id])
print(test_df.iloc[:,id])

In [ ]:
test_df

In [ ]:
a.write('Final_CRC.epw')

In [ ]:
from uwg import UWG

# Define the .epw, .uwg paths to create an uwg object.
epw_path = "EPW_Files/Final_CRC.epw" # available in resources directory.

# Initialize the UWG model by passing parameters as arguments, or relying on defaults
model = UWG.from_param_args(epw_path=epw_path, bldheight=11, blddensity=0.2169, 
                                vertohor=0.8, charlength=125, sensanth=0.835,
                                treecover=0.1028, grasscover=0.3084, vegstart=3, 
                                vegend=11, albveg=0.15, h_obs= 3, 
                                h_ubl1=700, windmin=7.339,
                                zone='7', nday=365)

#Campus Average:
#Building Density: 21.69%, 0.2169
#Grass Cover: 30.84%, 0.3084
#Tree Cover: 10.28%, 0.1028

#Total Vegetation: 41.11%

model.generate()
model.simulate()

# Write the simulation result to a file.
model.write_epw()


In [ ]:
r = epw()
r.read('EPW_Files/Final_CRC_UWG.epw')
result_df = r.dataframe
df_reset  = result_df.copy()

# Step 1: Reset index to move it to a column
df_reset = df_reset.reset_index()

# Step 2: Shift the DataFrame to the right by 1 column
df_shifted = df_reset.shift(1, axis=1)

# Step 3: Drop the last column which is now full of NaNs
df_shifted = df_shifted.iloc[:, :-1].drop("index", axis = 1)
df_shifted

In [ ]:
import matplotlib.pyplot as plt


hour = 6785

df_plot = df_shifted[hour:hour+24]

# Plot the temperature values for the hottest day
plt.figure(figsize=(12, 6))
plt.plot(df_shifted.index, df_shifted['Dry Bulb Temperature'], label='Dry Bulb Temperature', color='red')
plt.xlabel('Time')
plt.ylabel('Dry Bulb Temperature (°C)')
plt.title(f'Dry Bulb Temperature on the Hottest Day: {df_shifted}')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'epw_df' is your EPW DataFrame and it has an 'Hour' column and a 'Dry Bulb Temperature' column

# Step 1: Calculate Diurnal Averages for 'Dry Bulb Temperature'
diurnal_averages_temp = df_shifted.groupby('Hour')['Dry Bulb Temperature'].mean()

# Step 2: Print out the Diurnal Averages for 'Dry Bulb Temperature'
print("Diurnal Averages for Dry Bulb Temperature:")
print(diurnal_averages_temp)

# Step 3: Plot the Diurnal Averages for 'Dry Bulb Temperature'
diurnal_averages_temp.plot(kind='line', figsize=(10, 6), title='Diurnal Averages for Dry Bulb Temperature')
plt.xlabel('Hour of the Day')
plt.ylabel('Average Temperature (°C)')
plt.grid(True)
plt.show()


In [ ]:
Final_CRC = epw()
Final_CRC.read('EPW_Files/Final_CRC.epw')
CRC_df = Final_CRC.dataframe


In [ ]:
CRC_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'epw_df' is your first EPW DataFrame and 'CRC_Data' is the second one
# Both should have 'Hour' and 'Dry Bulb Temperature' columns

# Step 1: Calculate Diurnal Averages for 'Dry Bulb Temperature' for both DataFrames
diurnal_averages_epw = df_shifted.groupby('Hour')['Dry Bulb Temperature'].mean()
diurnal_averages_crc = CRC_df.groupby('Hour')['Dry Bulb Temperature'].mean()

# Step 2: Plot the Diurnal Averages for both DataFrames
plt.figure(figsize=(10, 6))
plt.plot(diurnal_averages_epw, label='EPW Data')
plt.plot(diurnal_averages_crc, label='Reference Data', linestyle='--')

# Step 3: Add titles and labels
plt.title('Comparison of Diurnal Averages for Dry Bulb Temperature')
plt.xlabel('Hour of the Day')
plt.ylabel('Average Temperature (°C)')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
diurnal_averages_simulated = df_shifted.groupby('Hour')['Dry Bulb Temperature'].mean()
diurnal_averages_original = CRC_df.groupby('Hour')['Dry Bulb Temperature'].mean()

mae = (diurnal_averages_original - diurnal_averages_simulated).abs().mean()
print("Mean Absolute Error (MAE):", mae)


In [ ]:
from uwg import UWG

# Define the .epw path to create a UWG object.
epw_path = "EPW_Files/Final_CRC.epw"

# Initialize the UWG model
model = UWG.from_param_args(epw_path=epw_path, bldheight=11, blddensity=0.0348, 
                            vertohor=0.8, charlength=125, sensanth=0.835,
                            treecover=0.1209, grasscover=0.3628, vegstart=3, 
                            vegend=11, albveg=0.15, h_obs=3, 
                            h_ubl1=700, windmin=7.339,
                            zone='7', nday=365)

#Science Square
#Building Density: 3.48%, 0.0348
#Grass Cover: 36.28%, 0.3628
#Tree Cover: 12.09%, 0.1209

#Total Vegetation: 48.37%


model.generate()
model.simulate()

# Write the simulation result to a file with a specified name.
model.write_epw()


In [ ]:
science_square = epw()
science_square.read('EPW_Files/Final_CRC_UWG.epw')
science_df = science_square.dataframe
sciencedf = science_df.copy()

In [ ]:
# Step 1: Reset index to move it to a column
sciencedf = sciencedf.reset_index()

# Step 2: Shift the DataFrame to the right by 1 column
science_shifted = sciencedf.shift(1, axis=1)

# Step 3: Drop the last column which is now full of NaNs
science_shifted = science_shifted.iloc[:, :-1].drop("index", axis = 1)

In [ ]:
# Ensure 'DateTime' column exists in df_shifted
if 'DateTime' not in df_shifted.columns:
    df_shifted['DateTime'] = pd.to_datetime(df_shifted[['Year', 'Month', 'Day', 'Hour', 'Minute']])

# Ensure 'DateTime' column exists in science_shifted
if 'DateTime' not in science_shifted.columns:
    science_shifted['DateTime'] = pd.to_datetime(science_shifted[['Year', 'Month', 'Day', 'Hour', 'Minute']])

# Define the date to compare
date_to_compare = pd.Timestamp('2023-04-06')

# Filter the data for the specified date
data_df_shifted = df_shifted[df_shifted['DateTime'].dt.date == date_to_compare.date()]
data_science_shifted = science_shifted[science_shifted['DateTime'].dt.date == date_to_compare.date()]

# Step 1: Calculate Diurnal Averages for 'Dry Bulb Temperature' for the specified date
diurnal_averages_df_shifted = data_df_shifted.groupby(data_df_shifted['DateTime'].dt.hour)['Dry Bulb Temperature'].mean()
diurnal_averages_science_shifted = data_science_shifted.groupby(data_science_shifted['DateTime'].dt.hour)['Dry Bulb Temperature'].mean()

# Step 2: Plot the Diurnal Averages for both DataFrames
plt.figure(figsize=(10, 6))

plt.plot(diurnal_averages_df_shifted, label=f'Reference: {date_to_compare.date()}', color='red')
plt.plot(diurnal_averages_science_shifted, label=f'Science Square: {date_to_compare.date()}', color='blue', linestyle='--')

# Step 3: Add labels and title
plt.xlabel('Hour of the Day')
plt.ylabel('Average Temperature (°C)')
plt.title(f'Comparison of Diurnal Averages on {date_to_compare.date()}')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
diurnal_averages_reference = df_shifted.groupby('Hour')['Dry Bulb Temperature'].mean()
diurnal_averages_science = science_shifted.groupby('Hour')['Dry Bulb Temperature'].mean()

mae = (diurnal_averages_reference - diurnal_averages_science).abs().mean()
print("Mean Absolute Error (MAE):", mae)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ensure 'DateTime' column exists in df_shifted
if 'DateTime' not in df_shifted.columns:
    df_shifted['DateTime'] = pd.to_datetime(df_shifted[['Year', 'Month', 'Day', 'Hour', 'Minute']])

# Ensure 'DateTime' column exists in science_df
if 'DateTime' not in science_df.columns:
    science_df['DateTime'] = pd.to_datetime(science_df[['Year', 'Month', 'Day', 'Hour', 'Minute']])

# Define the date to compare
date_to_compare = pd.Timestamp('2023-04-06')

# Filter the data for the specified date
data_df_shifted = df_shifted[df_shifted['DateTime'].dt.date == date_to_compare.date()]
data_science_df = science_df[science_df['DateTime'].dt.date == date_to_compare.date()]

# Create the plot
plt.figure(figsize=(10, 10))

# Plot the data for df_shifted with a solid line
plt.plot(data_df_shifted['DateTime'].dt.hour + data_df_shifted['DateTime'].dt.minute / 60, 
         data_df_shifted['Dry Bulb Temperature'], 
         label=f'DF Shifted: {date_to_compare.date()}', color='red', linestyle='-')

# Plot the data for science_df with a dashed line
plt.plot(data_science_df['DateTime'].dt.hour + data_science_df['DateTime'].dt.minute / 60, 
         data_science_df['Dry Bulb Temperature'], 
         label=f'Science DF: {date_to_compare.date()}', color='blue', linestyle='--')

# Add dots at x = 5, 10, 15, 20
for x in range(0, 24, 5):
    df_shifted_y = data_df_shifted.loc[data_df_shifted['DateTime'].dt.hour == x, 'Dry Bulb Temperature']
    science_df_y = data_science_df.loc[data_science_df['DateTime'].dt.hour == x, 'Dry Bulb Temperature']
    if not df_shifted_y.empty:
        plt.plot(x, df_shifted_y.values[0], 'ro')  # Red dot for df_shifted
    if not science_df_y.empty:
        plt.plot(x, science_df_y.values[0], 'bo')  # Blue dot for science_df

# Add labels and title
plt.xlabel('Hour of the Day')
plt.ylabel('Temperature (°C)')
plt.title(f'Temperature Profiles on {date_to_compare.date()}')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from uwg import UWG

# Define the .epw path to create a UWG object.
epw_path = "science.epw"

# Initialize the UWG model
model = UWG.from_param_args(epw_path=epw_path, bldheight=20, blddensity=0.2, 
                            vertohor=0.8, charlength=125, sensanth=0.835,
                            treecover=0.2, grasscover=0.6, vegstart=3, 
                            vegend=11, albveg=0.15, h_obs=3, 
                            h_ubl1=700, windmin=7.339,
                            zone='7', nday=365)

#original sciencec square inputs
#6.62 bld
#73.81 veg
#18.4525 tree
#55.3575 grass

#1st increment: 78.81: 0.5911 trees and 0.1970 grass and 0.0662
#2nd increment: 83.81: 0.6286 trees and 0.2095 grass and 0.0662
#3rd increment: 88.81: 0.6661 trees and 0.2220 grass and 0.0662


model.generate()
model.simulate()

# Write the simulation result to a file with a specified name.
model.write_epw()



In [ ]:
from uwg import UWG

# Define the .epw path to create a UWG object.
epw_path = "science2.epw"

# Initialize the UWG model
model = UWG.from_param_args(epw_path=epw_path, bldheight=11, blddensity=0.0662, 
                            vertohor=0.8, charlength=125, sensanth=0.835,
                            treecover=0.2095, grasscover=0.6286, vegstart=3, 
                            vegend=11, albveg=0.15, h_obs=3, 
                            h_ubl1=700, windmin=7.339,
                            zone='7', nday=365)

#original sciencec square inputs
#6.62 bld
#73.81 veg
#18.4525 tree
#55.3575 grass

#1st increment: 78.81: 0.5911 trees and 0.1970 grass and 0.0662
#2nd increment: 83.81: 0.6286 trees and 0.2095 grass and 0.0662
#3rd increment: 88.81: 0.6661 trees and 0.2220 grass and 0.0662


model.generate()
model.simulate()

# Write the simulation result to a file with a specified name.
model.write_epw()



In [ ]:
from uwg import UWG

# Define the .epw path to create a UWG object.
epw_path = "science3.epw"

# Initialize the UWG model
model = UWG.from_param_args(epw_path=epw_path, bldheight=11, blddensity=0.0662, 
                            vertohor=0.8, charlength=125, sensanth=0.835,
                            treecover=0.222, grasscover=0.6661, vegstart=3, 
                            vegend=11, albveg=0.15, h_obs=3, 
                            h_ubl1=700, windmin=7.339,
                            zone='7', nday=365)

#original sciencec square inputs
#6.62 bld
#73.81 veg
#18.4525 tree
#55.3575 grass

#1st increment: 78.81: 0.5911 trees and 0.1970 grass and 0.0662
#2nd increment: 83.81: 0.6286 trees and 0.2095 grass and 0.0662
#3rd increment: 88.81: 0.6661 trees and 0.2220 grass and 0.0662


model.generate()
model.simulate()

# Write the simulation result to a file with a specified name.
model.write_epw()



In [ ]:
from epw import epw
science1 = epw()
science1.read('science_UWG.epw')
science1df = science1.dataframe
science1dfdf = science1df.copy()

In [ ]:
science2 = epw()
science2.read('science2_UWG.epw')
science2df = science2.dataframe
science2dfdf = science2df.copy()

In [ ]:
science3 = epw()
science3.read('science3_UWG.epw')
science3df = science3.dataframe
science3dfdf = science3df.copy()